In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../src")

In [3]:
import numpy as np
from pydrake.all import (
    StartMeshcat,
    Simulator
)

In [4]:
from world import make_n_quadrotor_system
from util import DisableCollisionChecking
from stabilization import find_fixed_point_snopt, lqr_stabilize_to_point, add_controller_to_system

In [5]:
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7000


In [6]:
np.random.seed(1)
n_quadrotors = 5

In [7]:
diagram, plant = make_n_quadrotor_system(meshcat, n_quadrotors)

In [8]:
fixed_point, fixed_control = find_fixed_point_snopt(diagram)
print(fixed_point)
print(fixed_control)
fixed_point.shape

RuntimeError: Exception while evaluating SNOPT costs and constraints: 'RuntimeError: QuaternionToRotationMatrix(): All the elements in a quaternion are zero.

At:
  /home/tommy/Documents/homework/6.8210/project/repo/notebooks/../src/stabilization.py(74): time_derivative
  /home/tommy/Documents/homework/6.8210/project/repo/notebooks/../src/stabilization.py(80): find_fixed_point_snopt
  /tmp/ipykernel_126659/1449802733.py(1): <module>
  /home/tommy/Documents/homework/6.8210/env_underactuated/lib/python3.10/site-packages/IPython/core/interactiveshell.py(3460): run_code
  /home/tommy/Documents/homework/6.8210/env_underactuated/lib/python3.10/site-packages/IPython/core/interactiveshell.py(3400): run_ast_nodes
  /home/tommy/Documents/homework/6.8210/env_underactuated/lib/python3.10/site-packages/IPython/core/interactiveshell.py(3221): run_cell_async
  /home/tommy/Documents/homework/6.8210/env_underactuated/lib/python3.10/site-packages/IPython/core/async_helpers.py(129): _pseudo_sync_runner
 

In [ ]:
Q_quadrotor = [10.0] * 6 + [1.0] * 6
Q_freebody = [10.] * 13
Q = np.diag(Q_quadrotor * n_quadrotors + Q_freebody)
R = np.eye(4 * n_quadrotors)
lqr_controller = lqr_stabilize_to_point(diagram, fixed_point, fixed_control, Q, R)

controlled_diagram, controlled_plant = add_controller_to_system(diagram, lqr_controller)

In [ ]:
simulator = Simulator(controlled_diagram)
simulator.set_target_realtime_rate(1)
context = simulator.get_mutable_context()

sg = diagram.GetSubsystemByName("scene_graph")
DisableCollisionChecking(sg, context)

simulator.set_target_realtime_rate(0.1)
# Simulate
while True:
    context.SetTime(0.0)
    context.SetContinuousState(
        fixed_point
    )
    simulator.Initialize()
    simulator.AdvanceTo(1)

